In [ ]:
import pandas as pd
from wikiwho_chobj import Chobjer

context = 5
co = Chobjer(article="39570", pickles_path='../../bert', lang='en', context=context)
df = pd.DataFrame(co.iter_chobjs(), columns = next(co.iter_chobjs()).keys())

In [ ]:
import nltk
from nltk.corpus import stopwords 
nltk.download("stopwords")
stop_words = set(stopwords.words('english'))

In [ ]:
import sys
sys.path.insert(1, '../utils/')
from merge import combine

df=df[(df['ins_tokens_str'].str.len() + df['del_tokens_str'].str.len()) <= 20]
jlogie = pd.read_csv("../../John_Logie_Baird_FULL.csv")
merged = df.apply(lambda x: combine(x, jlogie), axis=1)

# captures if we also want to use changeobjects that do not have tokens that are ground-truth labelled
OUTER_JOIN = True

merged = merged.dropna(how="all")
if not OUTER_JOIN:
    merged = merged[(merged["birth_place"].isna() & merged["Bulk"].isna() & merged["nationality"].isna() & merged["Link"].isna())== False]

In [ ]:
import re
def making_list(tokens):
    new_tokens = []
    for item in tokens:
        new_tokens.append(' '.join(word for word in item if re.match("^[a-zA-Z0-9_]*$", word)))

    return(new_tokens)


all_tokens = []
merged = merged.reset_index()
#merged=merged.drop(['level_0'], axis=1)
for i, row in merged.iterrows():
    all_tokens.append(making_list([row['left_token_str'],row['right_token_str']]))#,row['ins_tokens_str'],row['del_tokens_str']]))
    merged.loc[i, 'left_token_str'] = all_tokens[i][0]
    merged.loc[i,'right_token_str'] = all_tokens[i][1]
    merged.loc[i,'ins_tokens_str'] = making_list([row['ins_tokens_str']])
    merged.loc[i,'del_tokens_str'] = making_list([row['del_tokens_str']])

In [ ]:
all_tokens

## DistilBert

In [ ]:
import torch
import numpy as np
from pytorch_transformers import (DistilBertConfig,
                                     DistilBertModel, 
                                     DistilBertTokenizer)
from pytorch_transformers.modeling_utils import SequenceSummary

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased',
                                    output_hidden_states=True,
                                    output_attentions=True)


In [ ]:
sequence_summary = SequenceSummary(model.config)
features = pd.DataFrame()
new_vector = []
for row in all_tokens:
    vector=[]
    for token in row:
        if token.isspace() or token == '':
            vector.append(np.full((768), 0))
        else:
            sen = torch.tensor([tokenizer.encode(token)])
            vector.append(sequence_summary(model(sen)[0])[0].detach().numpy())
    new_vector.append(np.concatenate((vector[0], vector[1])))
    
    #features = features.append(pd.DataFrame(np.concatenate((vector[0], vector[1]))).T)  


In [ ]:
print(len(new_vector[0]))

In [ ]:
sequence_summary = SequenceSummary(model.config)
new_vector = []
for row in all_tokens:
    vector=[]
    for token in row:
        if token.isspace() or token == '':
            vector.extend(np.full((768), 0))
        else:
            sen = torch.tensor([tokenizer.encode(token)])
            vector.extend(sequence_summary(model(sen)[0])[0].detach().numpy())
    #print(len(vector))
    new_vector.append(vector)

features = pd.DataFrame(np.row_stack(new_vector))


In [ ]:
features.shape

## DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
db = DBSCAN(eps=2, min_samples=4).fit(features)
labels = db.labels_

# number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
from sklearn.manifold import TSNE

X = TSNE(random_state=42).fit_transform(features)

In [ ]:
plot_data_dbscan = pd.concat([pd.DataFrame(X), pd.Series(labels), merged], axis=1)

plot_data_dbscan.columns = ['t-SNE-X', 't-SNE-Y', 'db_cluster', 'index','Bulk',
                  'Link',          'action',     'birth_place',
           'del_end_pos',   'del_start_pos',      'del_tokens',
        'del_tokens_str',          'editor',        'from_rev',
        'from_timestamp',     'ins_end_pos',   'ins_start_pos',
            'ins_tokens',  'ins_tokens_str',      'left_neigh',
            'left_token',  'left_token_str',     'nationality',
               'page_id',     'right_neigh',     'right_token',
       'right_token_str',            'text',          'to_rev',
          'to_timestamp',           'token']
plot_data_dbscan = plot_data_dbscan[plot_data_dbscan['db_cluster'] != -1].reset_index()
plot_data_dbscan.to_csv("plotData_distilbert_dbscan_eps2_samples4.csv")